In [ ]:
'''
Project description link: https://thecleverprogrammer.com/2023/07/31/music-recommendation-system-using-python/
# In the above link, there is project description. We can onluy 100 songs at a time donwload with  a given code in the project
# I edited the code so that unlimited number of song data can be downloaded using Spotify API.
# I downloaded 3000 song, and among them I offered a recommendation system that 
# recommends songs and artists according to popularity and danceability critearia
'''

In [42]:
import requests
import base64

# Replace with your own Client ID and Client Secret
CLIENT_ID = ''
CLIENT_SECRET = ''

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [16]:
#!pip install spotipy

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd

def retrieve_tracks(playlist_id, access_token,max_offset):
    
    df = pd.DataFrame()
    
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Initialize the offset parameter
    offset = 0

    # Create an empty list to store the playlist tracks
    playlist_tracks = []

    # Get the tracks from the playlist
    while offset <=(max_offset-100):
        
        playlist_tracks_chunk = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))', offset=offset)

        # -------------------
        
        # Extract relevant information and store in a list of dictionaries
        music_data = []
        
        for track_info in playlist_tracks_chunk['items']:

            track = track_info['track']
            track_name = track['name']
            artists = ', '.join([artist['name'] for artist in track['artists']])
            album_name = track['album']['name']
            album_id = track['album']['id']
            track_id = track['id']

            # Get audio features for the track
            audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

            # Get release date of the album
            try:
                album_info = sp.album(album_id) if album_id != 'Not available' else None
                release_date = album_info['release_date'] if album_info else None
            except:
                release_date = None

            # Get popularity of the track
            try:
                track_info = sp.track(track_id) if track_id != 'Not available' else None
                popularity = track_info['popularity'] if track_info else None
            except:
                popularity = None

            # Add additional track information to the track data
            track_data = {
                'Track Name': track_name,
                'Artists': artists,
                'Album Name': album_name,
                'Album ID': album_id,
                'Track ID': track_id,
                'Popularity': popularity,
                'Release Date': release_date,
                'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
                'Explicit': track_info.get('explicit', None),
                'External URLs': track_info.get('external_urls', {}).get('spotify', None),
                'Danceability': audio_features['danceability'] if audio_features else None,
                'Energy': audio_features['energy'] if audio_features else None,
                'Key': audio_features['key'] if audio_features else None,
                'Loudness': audio_features['loudness'] if audio_features else None,
                'Mode': audio_features['mode'] if audio_features else None,
                'Speechiness': audio_features['speechiness'] if audio_features else None,
                'Acousticness': audio_features['acousticness'] if audio_features else None,
                'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
                'Liveness': audio_features['liveness'] if audio_features else None,
                'Valence': audio_features['valence'] if audio_features else None,
                'Tempo': audio_features['tempo'] if audio_features else None,
                # Add more attributes as needed
            }
            music_data.append(track_data)
            
        df_chunk = pd.DataFrame(music_data)
        #-----------------------------------------------------------
        
        df = pd.concat([df, df_chunk])

        # Increment the offset parameter
        offset += 100
        print(offset)

    return df

In [41]:
#df = retrieve_tracks("5S8SJdl1BDc0ugpkEvFsIL", access_token,3000)

In [6]:
#df.to_csv("music_data.csv")

In [19]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('music_data.csv')

In [6]:
df

,Unnamed: 0,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,0,...Ready For It?,Taylor Swift,...Ready For It?,0HG8fMDhvN2tH5uPHFsyZP,7zgqtptZvhf8GEmdsM2vp2,0,2017-09-03,208198,False,...,0.779,2,-6.454,1,0.1350,0.0665,0.000000,0.1550,0.453,160.000
1,1,Life Changes,Thomas Rhett,Life Changes,4w5Jvreahp3yvLqc4vCr9I,4Vxu50qVrQcycjRyJQaZLC,63,2017-09-08,190227,False,...,0.845,7,-4.370,1,0.0576,0.1000,0.000000,0.0452,0.809,87.972
2,2,24K Magic,Bruno Mars,24K Magic,4PgleR09JVnm3zY1fW3XBA,6b8Be6ljOzmkOmFslEb23P,81,2016-11-17,225983,False,...,0.803,1,-4.282,1,0.0797,0.0340,0.000000,0.1530,0.632,106.970
3,3,Galway Girl,Ed Sheeran,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,0afhq8XCExXpqazXczTSve,80,2017-03-03,170827,False,...,0.876,9,-3.374,1,0.1000,0.0735,0.000000,0.3270,0.781,99.943
4,4,Photograph,Ed Sheeran,x (Deluxe Edition),1xn54DMo2qIqBuMqHtUsFd,1HNkqx9Ahdgi1Ixy2xkKkL,86,2014-06-21,258987,False,...,0.379,4,-10.480,1,0.0476,0.6070,0.000464,0.0986,0.201,107.989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,95,The Spectre,Alan Walker,The Spectre,1IKRstg3XuCuLWeCg3oaAW,2DGa7iaidT5s0qnINlwMjJ,76,2017-09-15,193788,False,...,0.711,1,-7.437,0,0.0383,0.0214,0.001910,0.0903,0.198,127.957
2996,96,Happier,"Marshmello, Bastille",Happier,78EicdHZr5XBWD7llEZ1Jh,2dpaYNEQHiRxtZbfNsse99,82,2018-08-17,214290,False,...,0.792,5,-2.749,1,0.0452,0.1910,0.000000,0.1670,0.671,100.015
2997,97,Drew Barrymore,Bryce Vine,Drew Barrymore,5PPDA9I1d0sl1NMbb7Pj5b,0OgGn1ofaj55l2PcihQQGV,63,2017-12-01,191147,True,...,0.574,11,-6.889,0,0.0572,0.4600,0.002720,0.1020,0.261,151.985
2998,98,Let You Love Me,Rita Ora,Let You Love Me,4INRJMYMJrSPsf5mbW26Fm,6xtcFXSo8H9BZN637BMVKS,60,2018-09-21,190000,False,...,0.854,10,-3.149,1,0.1980,0.2880,0.000000,0.0773,0.456,94.419


In [7]:
df.columns

Index(['Unnamed: 0', 'Track Name', 'Artists', 'Album Name', 'Album ID',
       'Track ID', 'Popularity', 'Release Date', 'Duration (ms)', 'Explicit',
       'External URLs', 'Danceability', 'Energy', 'Key', 'Loudness', 'Mode',
       'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness',
       'Valence', 'Tempo'],
      dtype='object')

In [8]:
# Remove the index
df = df.reset_index(drop=True)

In [9]:
df.drop(['Unnamed: 0'], axis =1 , inplace=True)

In [11]:
df.columns

Index(['Track Name', 'Artists', 'Album Name', 'Album ID', 'Track ID',
       'Popularity', 'Release Date', 'Duration (ms)', 'Explicit',
       'External URLs', 'Danceability', 'Energy', 'Key', 'Loudness', 'Mode',
       'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness',
       'Valence', 'Tempo'],
      dtype='object')

In [15]:
grouped = df.groupby('Popularity')
#grouped.sort_values('Popularity',ascending=False).head(5)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track Name        3000 non-null   object 
 1   Artists           3000 non-null   object 
 2   Album Name        3000 non-null   object 
 3   Album ID          3000 non-null   object 
 4   Track ID          3000 non-null   object 
 5   Popularity        3000 non-null   int64  
 6   Release Date      2999 non-null   object 
 7   Duration (ms)     3000 non-null   int64  
 8   Explicit          3000 non-null   bool   
 9   External URLs     3000 non-null   object 
 10  Danceability      3000 non-null   float64
 11  Energy            3000 non-null   float64
 12  Key               3000 non-null   int64  
 13  Loudness          3000 non-null   float64
 14  Mode              3000 non-null   int64  
 15  Speechiness       3000 non-null   float64
 16  Acousticness      3000 non-null   float64


In [38]:
#Most popular songs according to popularity and Danceability
grouped=df.groupby('Track Name').agg({ "Danceability":[np.size,np.sum,np.mean],"Popularity":[np.mean]})
grouped["Track Index"]=grouped["Danceability"]["sum"]*grouped["Popularity"]["mean"]
grouped['Track Index'].sort_values(ascending=False).head(5)

Track Name
Teenage Dream       176.874
California Gurls    170.424
Payphone            167.579
Call Me Maybe       163.647
Ice Ice Baby        162.848
Name: Track Index, dtype: float64

In [40]:
#Most popular artist according to popularity and Danceability
grouped=df.groupby('Artists').agg({ "Danceability":[np.size,np.sum,np.mean],"Popularity":[np.mean, np.sum]})
grouped["Artist index"]=grouped["Danceability"]["sum"]*grouped["Popularity"]["sum"]
grouped['Artist index'].sort_values(ascending=False).head(5)

Artists
Taylor Swift    676469.565
Ed Sheeran       17750.208
Post Malone      14158.496
Thomas Rhett     11396.420
Bruno Mars       10183.776
Name: Artist index, dtype: float64